# Variables

In [174]:
import torch
import os
from skimage import io
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
#--------------------------------------------------------------
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")


#model hyperparameters
model_output_size=4
BATCH_SIZE = 8
learning_rate=0.0001
train_epoch=20 #10

#EPOCH = 1000 #

transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485,0.456,0.406],
                                                                             std=[0.229,0.224,0.225])])



# DATA LOAD

In [175]:
class MyDataset(Dataset):
  def __init__(self, image_dir, label, transforms=None):
    self.image_dir = image_dir
    self.label = label
    self.image_list = os.listdir(self.image_dir)
    self.transforms = transforms
  
  def __len__(self):
    return len(self.image_list)
  
  def __getitem__(self,idx):
    # if torch.is_tensor(idx):
    #   idx = idx.tolist()

    image_name = os.path.join(self.image_dir, self.image_list[idx])
    image = io.imread(image_name)

    ### transform
    image = transforms(image)

    return (image,self.label)

root = '/kaggle/input/swdl2020' #'/kaggle/input/swdl2020': root project repository
#cheetah : 0 , jaguar : 1, tiger : 2, hyena : 3

cheetah_train = MyDataset(root+"/train/cheetah_train_resized",0,transforms)
jaguar_train = MyDataset(root+"/train/jaguar_train_resized",1,transforms)
tiger_train = MyDataset(root+"/train/tiger_train_resized",2,transforms)
hyena_train = MyDataset(root+"/train/hyena_train_resized",3,transforms)
train_set = ConcatDataset([cheetah_train, jaguar_train, tiger_train, hyena_train])
print("Number of Training set images : ", len(train_set))

cheetah_val = MyDataset(root+"/validation/cheetah_validation_resized",0, transforms)
jaguar_val = MyDataset(root+"/validation/jaguar_validation_resized",1, transforms)
tiger_val = MyDataset(root+"/validation/tiger_validation_resized",2, transforms)
hyena_val = MyDataset(root+"/validation/hyena_validation_resized",3,transforms)
val_set = ConcatDataset([cheetah_val, jaguar_val, tiger_val, hyena_val])
print("Numver of Validation set images : ", len(val_set))

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle = True) #params: train_set, 16, True
#cheetah_train_loader = DataLoader(cheeta_train_set, batch_size=BATCH_SIZE, shuffle = True)
#jaguar_train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle = True)
##tiger_train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle = True)
#hyena_train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle = True)

val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
#cheetah_val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
#jaguar_val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
#tiger_val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
#hyena_val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

Number of Training set images :  3600
Numver of Validation set images :  400


# MODEL

In [176]:
##### YOUR MODEL#####

class myResnet18(nn.Module):
  def __init__(self):
    super(myResnet18, self).__init__()

    #Define Convolution Operation(attributes)
    self.resnet = models.resnet18() 
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    self.fc = nn.Linear(in_features=1000, out_features=model_output_size, bias=True) #nn.Linear()

  def forward(self, x):
    input=x
    #x = x.unsqueeze(0) # [3, 400, 400] -> [1, 3, 400, 400]
    x = self.resnet(x) # [1, 3, 400, 400] -> [1, 1000]
    #print(x.shape)
    x = x.view(-1, 1000) # INPUT [1, 64, 16, 16] -> OUTPUT [1, 64*16*16]  #input텐서를 지정된 output텐서로 펴주는 과정. [1,16384]로.
    #print('model_x.view: ',x.shape)
    x = F.softmax(self.fc(x),dim=1)#torch.sigmoid(self.fc(x)) # INPUT [1, 64*16*16] -> OUTPUT [1, 1]
    #print('modelResult: ',x.shape) #sigmoid=1,1 ->torch.Size([16, 16])
    return x

model = myResnet18()
loss_func=nn.CrossEntropyLoss()


'''
#param variables
lr=0.001

resnetModel = Resnet()
loss_func=nn.CrossEntropyLoss()
#optimizer=torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
'''

x = torch.rand(3, 400, 400)
print('1- ',x.shape)
x=x.unsqueeze(0)
x=model.resnet(x)
print('2- ',x.shape)
x=x.view(-1,1000)
print('3- ',x.shape)
x=F.softmax(model.fc(x),dim=1)
print('4- ',x.shape)


1-  torch.Size([3, 400, 400])
2-  torch.Size([1, 1000])
3-  torch.Size([1, 1000])
4-  torch.Size([1, 4])


# TRAIN

In [177]:
from tqdm.notebook import tqdm # for visualize training step. iterative한 것의 진행도를 보여줌. 트레이닝 정도를 시각화. 

def train(model, train_loader, optimizer, epoch):
  model.train() #training mode
  #for batch_idx, (image, target) in enumerate(train_loader): #target=label
  for batch_idx, (image, target) in tqdm(enumerate(train_loader)):
    data, target = image.to(DEVICE), target.to(DEVICE)
    #print(data.shape) #torch.Size([16, 3, 400, 400])
    #print('target: ',target) #torch.Size([1, 16])
    optimizer.zero_grad()
    output = model(data).to(DEVICE) #cuda()
    #print(batch_idx)
    #print(output.shape)
    #print('output0: ',output)
    #output=torch.transpose(output,0,1)
    output=output.squeeze(1)
    #print(output.shape) #torch.Size([16, 16])
    #print(target.shape)
    #print('output0: ',output[0])
    loss = loss_func(output,target) #nn.CrossEntropyLoss(output,target) 
    loss.backward()
    optimizer.step()

    if batch_idx % 150 == 0 :
      print('Train Epoch : {} [{}/{} ({:.0f})%]\tLoss: {:.6f}'
      .format(epoch, batch_idx*len(image),len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item()))

def evaluate(model, test_loader):
  model.eval() #validation mode
  test_loss =0
  correct =0
  with torch.no_grad():
    #for (image, target) in test_loader:
    for (image, target) in tqdm(test_loader):
      image, label = image.to(DEVICE), target.to(DEVICE)
      output = model(image).to(DEVICE)

      test_loss += F.cross_entropy(output, label, reduction='sum').item()
      pred = output.max(1, keepdim=True)[1]
      correct+= pred.eq(label.view_as(pred)).sum().item()
  
  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy

# EXECUTE

In [178]:
#cpu model
#net=myCNN().to(DEVICE) #cuda/cpu()
#criterion=nn.BCELoss #Binary Cross Entropy Loss
#myOptimizer= torch.optim.Adam(net.parameters(), lr=0.001) #params: net.params(),0.001,

resnetModel=myResnet18().to(DEVICE)#
criterion=nn.BCELoss()#nn.CrossEntropyLoss() #Binary Cross Entropy Loss
optimizer= torch.optim.Adam(resnetModel.parameters(), lr=learning_rate) #params: net.params(),0.001,


#????
train_info=[]
test_info=[]
save_path='./ClassificationNetwork/'


#training epoches
for epoch in range(1, train_epoch+1):
  print('#---------------------------------------------')
  print('train epoch: {}'.format(epoch))
  train(resnetModel, train_loader, optimizer, epoch)
  #params: model(myCNN), DataLoader(train_set, 16, True), SGD(model.parameters(),lr=0.001,momentum=0.9) , EPOCH(1000)
  print('validation epoch: {}'.format(epoch))
  test_loss, test_accuracy = evaluate(resnetModel, val_loader)
  print('[epoch {} Result] Test Loss : {:.4f}, Accuracy : {:.4f}%'.format(epoch, test_loss, test_accuracy))

#---------------------------------------------
train epoch: 1


Train Epoch : 1 [0/3600 (0)%]	Loss: 1.389941
Train Epoch : 1 [1200/3600 (33)%]	Loss: 1.173731
Train Epoch : 1 [2400/3600 (67)%]	Loss: 1.060435

validation epoch: 1



[epoch 1 Result] Test Loss : 1.1162, Accuracy : 63.2500%
#---------------------------------------------
train epoch: 2


Train Epoch : 2 [0/3600 (0)%]	Loss: 1.150695
Train Epoch : 2 [1200/3600 (33)%]	Loss: 1.049751
Train Epoch : 2 [2400/3600 (67)%]	Loss: 0.966003

validation epoch: 2



[epoch 2 Result] Test Loss : 1.0248, Accuracy : 75.0000%
#---------------------------------------------
train epoch: 3


...


[epoch 18 Result] Test Loss : 0.8218, Accuracy : 93.2500%
#---------------------------------------------
train epoch: 19


Train Epoch : 19 [0/3600 (0)%]	Loss: 0.778937
Train Epoch : 19 [1200/3600 (33)%]	Loss: 0.754938
Train Epoch : 19 [2400/3600 (67)%]	Loss: 0.756723

validation epoch: 19



[epoch 19 Result] Test Loss : 0.8192, Accuracy : 93.0000%
#---------------------------------------------
train epoch: 20


Train Epoch : 20 [0/3600 (0)%]	Loss: 0.744209
Train Epoch : 20 [1200/3600 (33)%]	Loss: 0.764430
Train Epoch : 20 [2400/3600 (67)%]	Loss: 0.746419

validation epoch: 20



[epoch 20 Result] Test Loss : 0.8108, Accuracy : 93.7500%


# TEST LOADER

In [179]:
class TestDataset(Dataset):
  def __init__(self, image_dir, transforms=None):
    self.image_dir = image_dir
    self.image_list = os.listdir(self.image_dir)
    self.transforms = transforms
  
  def __len__(self):
    return len(self.image_list)
  
  def __getitem__(self,idx):
    # if torch.is_tensor(idx):
    #   idx = idx.tolist()

    image_name = os.path.join(self.image_dir, self.image_list[idx])
    image = io.imread(image_name)

    ### transform
    image = transforms(image)

    return (image,self.image_list[idx].split('.')[0])


test_set = TestDataset(root+"/test100", transforms) #test100
test_loader = DataLoader(test_set)

# PREDICTION

In [180]:
import pandas as pd

#cheetah : 0 , jaguar : 1, tiger : 2, hyena : 3
map = ['cheetah','jaguar','tiger','hyena']

resnetModel.eval()
df = pd.DataFrame(columns=['id','category'])
with torch.no_grad():
    #for (image, image_name) in test_loader:
    for (image, image_name) in tqdm(test_loader):
        image = image.to(DEVICE)
        output = resnetModel(image).to(DEVICE)
        pred = output.max(1, keepdim=True)[1]
        print('iter:')
        print('  1- ',image.shape)
        print('  2- ',image_name)
        print('  3- ',output.shape)
        print('  4- ',pred)
        
        df = df.append(pd.DataFrame([[image_name[0], map[pred.squeeze().tolist()]]], columns=['id','category']))
df #print df

iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('8966',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('938',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('5582',)
  3-  torch.Size([1, 4])
  4-  tensor([[1]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('7863',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('6666',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('6312',)
  3-  torch.Size([1, 4])
  4-  tensor([[0]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('10083',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('8129',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  (

iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('6876',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('3958',)
  3-  torch.Size([1, 4])
  4-  tensor([[1]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('6479',)
  3-  torch.Size([1, 4])
  4-  tensor([[1]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('6309',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('11976',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('3735',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('412',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('2276',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  (

,id,category
0,8966,hyena
0,938,hyena
0,5582,jaguar
0,7863,hyena
0,6666,hyena
...,...,...
0,13833,hyena
0,6189,tiger
0,1189,hyena
0,458,hyena


# SAVE CSV FILE

In [181]:
df.to_csv('/kaggle/working/'+'res.csv', index=False)
#df.to_csv(root+'/result.csv', index=False)